In [ ]:
from src.img import getBrightest, np, plotAB, getOffset, trim
from src.tools import group, xy, read, hough, parseData, templ
from matplotlib import pyplot as plt
import math as M
import cv2 as C

# file, sz, Y_AXIS, BEAD_FREQ, CT, BBOX, BEAD_SIZE, IBD, index = parseData()
file, sz, Y_AXIS, BEAD_FREQ, CT, BBOX, BEAD_SIZE, IBD, index = parseData()
(y0, x0, y1, x1) = BBOX

circle_mode = False
B_POS = (Y_AXIS, y0)
speed = BEAD_FREQ / IBD # how many px does the bead move per frame

REF = C.resize(read('frame.png'), (sz[0], sz[1]), interpolation=C.INTER_AREA)
file

In [ ]:
lums, cache = [], {}

for i in range(1, CT - 1):
  name = f'{i:03d}'
  img = read(f'./out_dir/{name}.png')
  cache[name] = np.sum(img > 200)
  lums.append(cache[name])
mean, std = np.mean(lums), np.std(lums)

for k in list(cache.keys()):
  if cache[k] < mean + std:
    del cache[k]

ranges = group(list(map(int, cache.keys())))
maxes = {}
for r in ranges:
  argmax = max(r, key=lambda k: cache.get(f"{k:03d}", 0))
  maxes[argmax] = cache[f"{argmax:03d}"]
print(ranges, maxes)

img = read(f'./in_dir/{max(maxes, key=maxes.get):03d}.png')
B_SIZ = round(img.shape[0] * 0.04)
x0, y0, x1, y1 = xy((B_POS[0], round(B_POS[1] + B_SIZ)), B_SIZ)
C.rectangle(img, (y1, x1), (y0, x0), 255, 2)

plt.imshow(img, cmap='gray')
plt.show()

In [ ]:
MOVE_REF, MOVE_MAIN = 0, 5
PAN_X, PAN_Y = 1, 0

REV = round(1 * BEAD_SIZE * speed)
print(max(maxes, key=lambda k: maxes[k]), REV)
bead = max([max(maxes, key=lambda k: maxes[k]) - REV, 1]) + MOVE_REF
print(f'./data/Kani_2/{file}', f"Using frame {bead}")

img_ref = read(f'./in_dir/{bead:03d}.png')
x0, y0, x1, y1 = xy((B_POS[0], round(B_POS[1] + B_SIZ)), B_SIZ)
img_bead = img_ref[x0+PAN_Y:x1+PAN_Y, y0+PAN_X:y1+PAN_X]

try:
  base = MOVE_MAIN + (bead - BEAD_FREQ if BEAD_FREQ < bead else bead + BEAD_FREQ)
  img_ref = read(f'./in_dir/{base:03d}.png')
  x0, y0, x1, y1 = xy((B_POS[0], round(B_POS[1] + B_SIZ)), B_SIZ)
  img_base = img_ref[x0+PAN_Y:x1+PAN_Y, y0+PAN_X:y1+PAN_X]
  plotAB(img_bead, img_base, f"Ref Frame:{bead}", f"BEAD Frame:{base}")
  plt.show()
except:
  print(f"Could not find frame {base}\nFolder has {CT} frames")
  print(f"BEAD_FREQ is {BEAD_FREQ} and current frame is {bead}")

In [ ]:
dy, dx = REF.shape
rs = [dx / 2, dy / 2, img_bead.shape[0] / 2, img_bead.shape[1] / 2, img_base.shape[0] / 2, img_base.shape[1] / 2]
rs = [round(r) for r in rs]
r_ = min(rs)
sy, sx = sz[:2]

if circle_mode:
  rme, rpe = r_ - 3, r_ + 3
  x0, y0, r0 = hough(img_bead, minDist=1, param1=150, param2=10, minRadius=rme, maxRadius=rpe)[0][0]
  img_bead2 = img_bead[y0-r0:y0+r0, x0-r0:x0+r0].copy()

  x1, y1, r1 = hough(img_base, minDist=1, param1=150, param2=10, minRadius=rme, maxRadius=rpe)[0][0]
  img_base2 = img_base[y1-r1:y1+r1, x1-r1:x1+r1].copy()

else:
  res_1 = templ(REF, img_bead)
  res_2 = templ(REF, img_base)
  R1 = (res_1[3][0] + 2, res_1[3][1], res_1[3][0] + sy, res_1[3][1] + sx)
  R2 = (res_2[3][0] + 2, res_2[3][1], res_2[3][0] + sy, res_2[3][1] + sx)

  img_bead2 = img_bead[R1[1]:R1[3], R1[0]:R1[2]]
  img_base2 = img_base[R2[1]:R2[3], R2[0]:R2[2]]

plotAB(img_bead2, img_base2, f"Ref Fr:{bead} {img_bead2.shape}", f"BEAD Fr:{base} {img_base2.shape}")
plt.savefig(f'./frames/{file}.png', bbox_inches='tight')
plt.show()

In [ ]:
Ty = getOffset(np.mean(img_bead2, axis=1), np.mean(img_base2, axis=1))
Tx = getOffset(np.mean(img_bead2, axis=0), np.mean(img_base2, axis=0))
print(f"Offset: {Ty}, {Tx}")

img_base2 = np.roll(np.roll(img_base2, Ty, axis=0), Tx, axis=1)
img_bead2, img_base2 = trim(img_bead2, img_base2)
img_bead2 = C.GaussianBlur(img_bead2, (5, 5), 0.1)
img_base2 = C.GaussianBlur(img_base2, (5, 5), 0.1)

plotAB(img_bead2, img_base2, f"Ref Frame:{bead} ({img_bead2.shape})", f"BEAD Frame:{base} ({img_base2.shape})")
plt.show()

In [ ]:
FLIP = False

diff = img_base2 - (img_bead2 * 0.9)
if FLIP: diff = 255 - diff

mdp = getBrightest(3, diff, limited=True)
bx, by = mdp

cx, cy = round(img_base2.shape[0] / 2), round(img_base2.shape[1] / 2)
plt.imshow(diff, cmap='gray')
plt.plot([cy, mdp[1]], [cx, mdp[0]], 'r')
plt.plot(cy, cx, 'ro')
plt.plot(mdp[1], mdp[0], 'bo')
plt.show()

D = M.sqrt((cx - mdp[0])**2 + (cy - mdp[1])**2)
theta = 180 - (M.atan2(cx - bx, by - cy) * 180 / M.pi)

# I did all calc with the axes swapped
print(f"Index|{BEAD_SIZE:.0f}px | {IBD:.1f}R | {D:.1f}px | {theta:.1f}° |")
string = f"\n{index}, {file}, {IBD:.1f}, {BEAD_SIZE:.1f}, {D:.0f}, {theta:.1f}"
print(string)

In [144]:
# filename, IDD, droplet radius, bead r, bead theta
with open("./dump.csv", 'a') as f:
  f.write(string)